In [1]:
# This part you need to run once per project, which I have already done, so it is commented out. 
# It imports necessary databases (possibly a lengthy process), and backs up the project directory 
# so that it is saved in the home directory. The backed up project can be used by other project 
# collaborators without the need to import any databases again. The next section starts with restoring 
# the saved project from directory.

# import bw2data as bd
# import bw2calc as bc
# import bw2io as bi
# import numpy as np

# ei_path = "/Users/akim/Documents/LCA_files/ecoinvent_38_cutoff/datasets"
# ei_name = "ecoinvent 3.8 cutoff"

# bd.projects.set_current("Building paper")

In [2]:
# # Import biosphere
# bi.bw2setup()

In [3]:
# # Import ecoinvent
# if ei_name in bd.databases:
#     print("database already exists")
# else:
#     ei = bi.SingleOutputEcospold2Importer(ei_path, ei_name)
#     ei.apply_strategies()
#     assert ei.all_linked
#     ei.write_database()

In [4]:
# bi.backup_project_directory("Building paper")

In [3]:
import os
os.chdir("/home/shinder/0_work/1_Models/2_building/2a_integrations/model_building/building-model/model_lca")

import bw2data as bd
import bw2calc as bc
import bw2io as bi
import numpy as np

# Only need to run this once! Otherwise you might work on this project, eg import `building lci` database, 
# but then overwrite it if you restore it from the backup
# fp_building_bw_project = "brightway2-project-Building paper-backup.19-September-2022-03-55PM.tar.gz"
# bi.restore_project_directory(fp_building_bw_project)

In [4]:
bd.projects.set_current("Building paper")
bd.databases

Databases dictionary with 3 object(s):
	biosphere3
	building lci
	ecoinvent 3.8 cutoff

In [11]:
# Import foreground database from excel

# delete database 
# -- doesn't hurt to do it every time you run the code, because it is very fast, but your database is then always updated
# del bd.databases["building lci"]  

# if "building lci" not in bd.databases:
fp_building_lci_excel = "building_lci_2.xlsx"
bld = bi.ExcelImporter(fp_building_lci_excel)
bld.apply_strategies()
bld.match_database("ecoinvent 3.8 cutoff", fields=("name", "location"))
bld.match_database()

if bld.all_linked:
    bld.write_database()
else:
    bld.statistics()

Extracted 2 worksheets in 0.04 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 15.40 seconds
Applying strategy: link_iterable_by_fields


Writing activities to SQLite3 database:


Applying strategy: link_iterable_by_fields
12 datasets
93 exchanges
0 unlinked exchanges
  


0% [############] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 11/24/2022 13:07:57
  Finished: 11/24/2022 13:07:57
  Total time elapsed: 00:00:00
  CPU %: 170.30
  Memory %: 0.08
Created database: building lci


In [123]:
list (bld.unlinked)

[]

In [13]:
# LCIA
bld = bd.Database("building lci")

n =12 #number of buildings
for x in range(1,n+1):
    building_name = str("building ")+str(x)
    demand_act = [act for act in bld if building_name in act['name']]
#     assert len(demand_act) == 1
    demand_act = demand_act[0]

    method = ('IPCC 2013', 'climate change', 'GWP 100a')

    lca = bc.LCA({demand_act: 1}, method)
    lca.lci()
    lca.lcia()
    print(building_name, lca.score)

NonsquareTechnosphere: Technosphere matrix is not square: 19577 activities (columns) and 19565 products (rows). Use LeastSquaresLCA to solve this system, or fix the input data

In [17]:
x = "wood pellet"
ei =    bd.Database("ecoinvent 3.8 cutoff")
demand_act = [act for act in ei if x in act['name'] and 'CH' in act['location']]
method = ('IPCC 2013', 'climate change', 'GWP 100a')
demand_act = demand_act[0]
print(demand_act)
lca = bc.LCA({demand_act: 1}, method)
lca.lci()
lca.lcia()
print(lca.score)

'heat production, wood pellet, at furnace 25kW' (megajoule, CH, None)
0.013365199703608386


In [ ]:
heat production, borehole heat exchanger, brine-water heat pump 10kW
heat, from municipal waste incineration to generic market for heat district or industrial, other than natural gas
market for heat, central or small-scale, natural gas
